In [ ]:
import tensorflow
from tensorflow import keras
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import numpy as np

# Loading Data Set
Three sets are created: training, validation, and test. 
- Labels are generated based on the folder structure.
- Loading in batches, of size 32, to reduce memory usage.
- Label mode is set to categorical, which means that the labels are encoded as a categorical vector.


The image load documentation is available [here](https://keras.io/api/data_loading/image/).

In [ ]:
train = keras.utils.image_dataset_from_directory(
    directory='ds_splitted/train/', # must be updated
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))

validation = keras.utils.image_dataset_from_directory(
    directory='ds_splitted/val/', # must be updated
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))
test = keras.utils.image_dataset_from_directory(
    directory='ds_splitted/test/', # must be updated
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))

# Visualizing the Data

In [ ]:
# histogram of class distribution in the data set
# remove underscores from class names
class_names = train.class_names
class_names = [name.replace('_', ' ') for name in class_names]
class_counts = [0]*len(class_names)

# get the number of samples in each class
for images, labels in train:
    for i in range(len(labels)):
        class_counts[np.argmax(labels[i])] += 1
# plot the histogram
plt.bar(class_names, class_counts)
plt.xticks(rotation=90)
plt.show()

# Creating the model

In [ ]:
conv_base = keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    pooling='max',
    input_shape=(256, 256, 3),
)
for layer in conv_base.layers:
    layer.trainable = False

top_model = conv_base.output
top_model = Flatten(name='flatten')(top_model)
top_model = Dense(4096, activation='relu')(top_model)
top_model = Dense(1072, activation='relu')(top_model)
top_model = Dropout(0.2)(top_model)
output_layer = Dense(12, activation='softmax')(top_model)

# Add callbacks

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',
                          patience=10,
                          restore_best_weights=True,
                          mode='min') # !TODO: must be configured properly

In [ ]:
tensorboard_callback = TensorBoard(log_dir='./logs') # !TODO: must be configured properly

# Compile model

In [ ]:
model = Model(inputs=conv_base.input, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train,
                    epochs=10,
                    validation_data=val,
                    callbacks=[early_stop, tensorboard_callback])